In [30]:
import pandas as pd
import numpy as np
from scipy.stats.contingency import chi2_contingency
from scipy.stats import chi2
from tqdm import tqdm
import plotly.express as px

# Funções e dicionários

In [25]:
z_dict = {99:   2.575,
          95:   1.960,
          90:   1.645,
          70:   1.035,
          67.5: 0.985}

In [20]:
def classificacao_nps(x):
    if x <= 6:
        return 'detrator'
    if x <= 8:
        return 'neutro'
    if x > 8:
        return 'promotor'
    else:
        return None

In [ ]:
def tabela_esperado(df_, colunas, filtros = None):
    
    if filtros is not None:
        
        df = df_.copy()
    
        for k, v in filtros.items():
            df = df[df[k].isin(v)]
            
    else:
        df = df_.copy()

    chi_sqr = df[colunas + ['NPS_GERAL']].copy()
    chi_sqr['class_nps'] = chi_sqr['NPS_GERAL'].apply(classificacao_nps)
    chi_sqr = chi_sqr[colunas + ['class_nps']].value_counts().reset_index(name = 'qtd')

    return chi_sqr

def relatorio_qui_quadrado(col1, col2, filtros = None, print_relatorio = True):

    base_esperado = tabela_esperado(colunas = [col1, col2], filtros = filtros)
    
    tabela_cat_1 = pd.DataFrame()

    for cat1 in base_esperado[col1].unique():

        if print_relatorio:
            print(cat1.upper())
            print('-'* 100)

        tabela_contingencia = []

        for cat2 in base_esperado[col2].unique():
            lista = list(base_esperado[(base_esperado[col1] == cat1) & (base_esperado[col2] == cat2)]['qtd'])

            tabela_contingencia.append(lista)

        tabela_contingencia = np.array(tabela_contingencia)

        # Aplicando o teste qui-quadrado
        alpha = 0.05
        chi2_stat, p, dof, expected = chi2_contingency(tabela_contingencia)
        valor_critico = chi2.ppf(1 - alpha, dof)
        
        if print_relatorio:

            # Exibindo os resultados
            print(f"Qui-quadrado: {chi2_stat:.2f}")
            print(f"Valor crítico: {valor_critico:.3f}")
            print(f"P-valor: {p:.4f}")
            print(f"Graus de liberdade: {dof}")
            print()
            print("Frequências observadas:")
            print(tabela_contingencia)
            print()
            print("Frequências esperadas:")
            print(expected.round())
            print()
            print("Observado - esperado:")
            print(tabela_contingencia - expected.round())
            print()

            # Interpretação
            print('Resultado:')
            if p < alpha:
                print("Rejeitamos a hipótese nula: há uma associação significativa entre os grupos e as categorias.")
            else:
                print("Não rejeitamos a hipótese nula: não há evidências de associação significativa entre os grupos e as categorias.")

            print()
        
        tmp = {col1:[cat1],
               'qui_quadrado':[f"{chi2_stat:.2f}"],
               'valor_critico':[f"{valor_critico:.3f}"],
               'p_valor':[f"{p:.4f}"]}
        
        df_tmp = pd.DataFrame(data = tmp)
        
        tabela_cat_1 = pd.concat([tabela_cat_1, df_tmp])
        
    return tabela_cat_1

def margem_erro(p, n, d, z):

    total = p + n + d

    nps = (p - d) / total 

    var_nps = ((1 - nps)**2 * (p/total)) + ((0 - nps)**2 * (n/total)) + ((1 - nps)**2 * (d/total))

    margem_erro = round(z * (var_nps/total)**(1/2) * 100, 1)
    
    return margem_erro

def margem_erro_grupo(df, z):
    
    classificacoes = ['promotor', 'neutro', 'detrator']
    valores = {classe: 0 for classe in classificacoes}
    
    for classe in classificacoes:
        if classe in df.class_nps.unique():
            valores[classe] = df[df['class_nps'] == classe].reset_index()['qtd'][0]
    
    margem = margem_erro(p = valores['promotor'], n = valores['neutro'], d = valores['detrator'], z = z)
    
    return margem


# Gerar tabela de NPS usando bootstrap com repetição

In [ ]:
# Bootstrap com repetição
df_margem_erro = pd.DataFrame()

for plataforma in ['Controle', 'Pós-pago']:
    for cat in ['E-mail', 'WhatsApp', 'Correios', 'Aplicativo']:
        for multa in ['Teve multa', 'Não teve multa']:
            
            print(f"{plataforma} | {cat} | {multa}")
            
            df_tmp = base_analitica_nps[(base_analitica_nps['COD_PLAT'] == plataforma) & 
                                        (base_analitica_nps['FLG_MULTA_ULTIMOS_12_MESES'] == multa) &
                                        (base_analitica_nps['DSC_CATEGORIA_IMPRESSAO'] == cat)]\
                .copy().reset_index(drop = True)


            for i in tqdm(range(10000)):

                amostra_i = np.random.randint(low = 0, high = df_tmp.shape[0], size = df_tmp.shape[0]).tolist()
                df_i = df_tmp.loc[amostra_i].copy()
                df_i['class_nps'] = df_i['NPS_GERAL'].apply(classificacao_nps)

                df_nps = df_i['class_nps'].value_counts()

                total = df_nps.sum()
                p = df_nps['promotor'] * 100 / total
                d = df_nps['detrator'] * 100 / total

                nps = round(p - d)

                tmp = pd.DataFrame(data = {'COD_PLAT':[plataforma],
                                           'DSC_CATEGORIA_IMPRESSAO':[cat],
                                           'FLG_MULTA_ULTIMOS_12_MESES':[multa],
                                           'nps': [nps]})

                df_margem_erro = pd.concat([df_margem_erro, tmp])

# Calcular margem de erro na própria tabela

In [18]:
base_nps = pd.read_csv('dados/NPStimeseries.csv')
base_nps['Survey date'] = pd.to_datetime(base_nps['Survey date'], format = '%d/%m/%Y')
base_nps['Month'] = base_nps['Survey date'].astype(str).apply(lambda x: x[:7])
# base_nps[['Month']].value_counts().reset_index().sort_values('Month')
base_nps.head(3)

,ID,Market,Survey date,Customer Name,Month,Quarter,NPS
0,1000,US,2021-09-01,Krista Richards,2021-09,3,10
1,1001,MEX,2021-11-07,Monica King,2021-11,4,9
2,1002,UK,2021-12-25,Ricky Armstrong,2021-12,4,0


In [28]:
def base_margem_erro(base_nps, coluna_nps, colunas_group, conf = 95):

    z_dict = {
        99:   2.575,
        95:   1.960,
        90:   1.645,
        70:   1.035,
        67.5: 0.985}

    tmp = base_nps.copy()
    tmp['class_nps'] = tmp[coluna_nps].apply(classificacao_nps)

    tmp = tmp[colunas_group + ['class_nps']].value_counts().reset_index(name = 'qtd')
    tmp['total'] = tmp.groupby(colunas_group)['qtd'].transform(sum)
    tmp['pct'] = round(tmp['qtd'] * 100 / tmp['total'])
    tmp = tmp.pivot(index = colunas_group, columns = 'class_nps', values = ['pct', 'qtd']).reset_index()
    tmp = tmp.fillna(0)
    tmp['nps'] = tmp[('pct', 'promotor')] - tmp[('pct', 'detrator')]
    tmp.columns = ['_'.join(map(str, col)).strip('_') for col in tmp.columns.values]
    tmp = tmp.merge(base_nps[colunas_group].value_counts().reset_index(name = 'volume'))
    tmp['nps_me'] = (tmp['qtd_promotor'] - tmp['qtd_detrator']) / tmp['volume']
    tmp['var_nps'] = tmp.apply(lambda row: ((1 - row['nps_me'])**2 * (row['qtd_promotor']/row['volume'])) + \
        ((0 - row['nps_me'])**2 * (row['pct_neutro']/row['volume'])) + \
        ((1 - row['nps_me'])**2 * (row['qtd_detrator']/row['volume'])), axis = 1)
    tmp['margem_erro'] = round(z_dict[conf] * (tmp['var_nps']/tmp['volume'])**(1/2) * 100, 1)
    tmp = tmp[colunas_group + ['volume', 'nps', 'margem_erro']]
    # tmp['text'] = '<b>' + tmp['nps'].round().astype(str).apply(lambda x: x.split('.')[0]) + '</b> (' + tmp['volume'].apply(sep_milhar) + ')'
    tmp['LI'] = round(tmp['nps'] - tmp['margem_erro'])
    tmp['LS'] = round(tmp['nps'] + tmp['margem_erro'])
    # tmp['volume_text'] = tmp['volume'].apply(sep_milhar)
    tmp = tmp[tmp['volume'] > 5]

    return tmp

In [32]:
base_me = base_margem_erro(base_nps = base_nps, coluna_nps = 'NPS', colunas_group = ['Market', 'Month'], conf = 95)
base_me.head()

C:\Users\Leticia\AppData\Local\Temp\ipykernel_15148\1943162473.py:14: FutureWarning:

The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.



,Market,Month,volume,nps,margem_erro,LI,LS
0,MEX,2021-01,131,14.0,12.8,1.0,27.0
1,MEX,2021-02,144,5.0,13.8,-9.0,19.0
2,MEX,2021-03,131,20.0,12.7,7.0,33.0
3,MEX,2021-04,134,5.0,13.9,-9.0,19.0
4,MEX,2021-05,145,33.0,10.0,23.0,43.0


### Gráfico

In [57]:
fig = px.line(base_me.sort_values(['Market', 'Month'], ascending = [False, False]),
             x = 'Month',
             y = 'nps',
             color = 'Market',
             facet_row = 'Market',
             text = 'nps',
             markers = True,
             custom_data = ['Month', 'margem_erro', 'LI', 'LS', 'volume'],
             error_y = 'margem_erro')
fig.update_layout(plot_bgcolor = 'white', height = 800)
fig.update_xaxes(title = '', tickfont = dict(size = 10, color = 'gray'))
fig.update_yaxes(title = '')
fig.update_traces(error_x_color = 'lightgray',
                  hovertemplate =
                      "%{customdata[0]}<br>" +
                      "<b>NPS:</b> %{y} (%{customdata[2]} ↔ %{customdata[3]})<br>" +
                      "<b>Qtd respondentes:</b> %{customdata[4]}")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.add_hline(y=0, line_dash="solid", line_color="black", line_width = 0.8)
fig.show()